In [1]:
!git clone https://github.com/sorki/python-mnist
!./python-mnist/bin/mnist_get_data.sh
!pip3 install emnist keras-tuner
from emnist import extract_training_samples

print("Imported the EMNIST libraries we need!")

fatal: destination path 'python-mnist' already exists and is not an empty directory.
data directory already present, exiting
Imported the EMNIST libraries we need!


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.utils import to_categorical
from emnist import extract_training_samples, extract_test_samples


from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

import kerastuner

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Mar 19 11:01:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    45W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
train_X, train_y = extract_training_samples("digits");
test_X, test_y = extract_test_samples("digits");

# Normalise
train_X = train_X.astype('float32')
train_X /= 255
test_X = test_X.astype('float32')
test_X /= 255

#train_X, val_X, train_y, val_y = train_test_split(images, labels, test_size= 0.10, random_state=42)


In [5]:
# Reshape image for CNN
train_X = train_X.reshape(-1, 28, 28, 1)
test_X = test_X.reshape(-1, 28, 28, 1)

num_classes = np.unique(train_y)
train_y = to_categorical(train_y)
test_y = to_categorical(test_y)

In [ ]:
from kerastuner.applications import HyperXception
from kerastuner.tuners import Hyperband

hypermodel = HyperXception(input_shape=(28, 28, 1), classes=10)

tuner = Hyperband(
    hypermodel,
    objective='val_accuracy',
    max_epochs=20,
    directory='my_dir',
    project_name='HyperXception')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(train_X, train_y,
             epochs=20,
             validation_split=0.2,
             callbacks=[stop_early])

Trial 26 Complete [00h 36m 17s]
val_accuracy: 0.9876041412353516

Best val_accuracy So Far: 0.9946458339691162
Total elapsed time: 09h 37m 40s

Search: Running Trial #27

Hyperparameter    |Value             |Best Value So Far 
activation        |relu              |relu              
conv2d_num_filters|32                |32                
kernel_size       |5                 |3                 
initial_strides   |2                 |2                 
sep_num_filters   |640               |256               
num_residual_bl...|5                 |3                 
pooling           |avg               |max               
num_dense_layers  |2                 |1                 
dropout_rate      |0.1               |0.5               
dense_use_bn      |1                 |1                 
learning_rate     |0.001             |0.001             
tuner/epochs      |20                |20                
tuner/initial_e...|0                 |7                 
tuner/bracket     |0           